In [1]:
# imports
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from fiona.crs import from_epsg
import shapely as shp
import seaborn 
import os
import geopandas as gpd
from fiona.crs import from_epsg
import urllib
import requests
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Pulling Data via API
wifi_data = pd.read_json("http://wifindproject.com/wifipulling/?device_model=VS425")

In [3]:
wifi_data.head()

,acc,altitude,app_version,bssid,caps,device_mac,device_model,droid_version,freq,idx,lat,level,lng,ssid,time
0,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:33,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629534,40.690966,-60,-73.987686,nyu-legacy,1497296993826
1,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:32,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629535,40.690966,-59,-73.987686,eduroam,1497296993826
2,19.957,0.0,0.3.3,cc:d5:39:27:f2:43,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629536,40.690966,-66,-73.987686,nyu-legacy,1497296993826
3,19.957,0.0,0.3.3,cc:d5:39:27:f2:42,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629537,40.690966,-67,-73.987686,eduroam,1497296993826
4,19.957,0.0,0.3.3,00:a2:ee:03:e3:20,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629538,40.690966,-73,-73.987686,constitution,1497296993826


In [4]:
# filter records with accuaracy scores of 50 and above
wifi_data = wifi_data.drop(wifi_data[wifi_data.acc > 50].index)

In [5]:
wifi_data.head()

,acc,altitude,app_version,bssid,caps,device_mac,device_model,droid_version,freq,idx,lat,level,lng,ssid,time
0,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:33,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629534,40.690966,-60,-73.987686,nyu-legacy,1497296993826
1,19.957,0.0,0.3.3,1c:e8:5d:ca:8f:32,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2437,56629535,40.690966,-59,-73.987686,eduroam,1497296993826
2,19.957,0.0,0.3.3,cc:d5:39:27:f2:43,[wpa-eap-tkip][wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629536,40.690966,-66,-73.987686,nyu-legacy,1497296993826
3,19.957,0.0,0.3.3,cc:d5:39:27:f2:42,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629537,40.690966,-67,-73.987686,eduroam,1497296993826
4,19.957,0.0,0.3.3,00:a2:ee:03:e3:20,[wpa2-eap-ccmp][ess],d3a2898a5e731607,VS425,5.1.1,2412,56629538,40.690966,-73,-73.987686,constitution,1497296993826


# Needed variables:
* Total Population
* Population Density
* House Hold Density
* Median Household Income
* Wifi Counts (open and other)
* Poverty Level
* Section 8 (T/F)
* Location (lat, long)
    

In [338]:
# 2010 & 2000 pop by census tract - BK
NYC_pop_data = pd.read_json("https://data.cityofnewyork.us/resource/rx2e-87id.json", dtype={'census_tract': str})

In [339]:
NYC_pop_data_2010.shape

(1000, 6)

In [340]:
NYC_pop_data.drop(NYC_pop_data[NYC_pop_data.year == 2000].index, inplace=True)

In [341]:
NYC_pop_data.head()

,borough,census_tract,dcp_borough_code,fips_county_code,population,year
761,Brooklyn,000100,3,47,4338,2010
762,Brooklyn,000200,3,47,1470,2010
763,Brooklyn,000301,3,47,4259,2010
764,Brooklyn,000501,3,47,4247,2010
765,Brooklyn,000502,3,47,2512,2010


In [342]:
NYC_pop_data.shape

(239, 6)

In [343]:
NYC_pop_data["boro_ct2010"] = NYC_pop_data.dcp_borough_code.astype(str) + NYC_pop_data.census_tract.astype(str)

In [344]:
NYC_pop_data["boro_ct2010"] = NYC_pop_data["boro_ct2010"].astype(int)

In [345]:
NYC_pop_data["boro_ct2010"].dtype

dtype('int64')

In [346]:
NYC_pop_data.borough.unique()

array(['Brooklyn'], dtype=object)

In [347]:
# 6 digits
# leading with 2 zeros for 2 digit numbers
# leading with 1 zeros for 3 digit numbers
# ending with 2 zeros for decimal places

NYC_pop_data[NYC_pop_data.census_tract.str.contains("0")].head()

,borough,census_tract,dcp_borough_code,fips_county_code,population,year,boro_ct2010
761,Brooklyn,000100,3,47,4338,2010,3000100
762,Brooklyn,000200,3,47,1470,2010,3000200
763,Brooklyn,000301,3,47,4259,2010,3000301
764,Brooklyn,000501,3,47,4247,2010,3000501
765,Brooklyn,000502,3,47,2512,2010,3000502


In [348]:
NYC_pop_data.shape

(239, 7)

In [349]:
# 2010 Census Tracts - NYC Open Data
# https://data.cityofnewyork.us/City-Government/2010-Census-Tracts/fxpq-c8ku

"""REFERENCE SYSTEM IDENTIFIER
* VALUE 2263
* CODESPACE EPSG
* VERSION 8.6.2"""

data = requests.get("https://data.cityofnewyork.us/resource/i69b-3rdj.json")
gdf = gpd.GeoDataFrame(data.json())
gdf.head()

,boro_code,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,the_geom
0,5,5000900,Staten Island,I,000900,9,SI22,West New Brighton-New Brighton-St. George,3903,2497009.69813,7729.01679376,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
1,1,1009800,Manhattan,I,009800,98,MN19,Turtle Bay-East Midtown,3808,1906016.35002,5534.19981063,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,1,1010000,Manhattan,I,010000,100,MN19,Turtle Bay-East Midtown,3808,1860938.37721,5692.16873705,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
3,1,1010200,Manhattan,I,010200,102,MN17,Midtown-Midtown South,3807,1860992.70485,5687.80243866,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
4,1,1010400,Manhattan,I,010400,104,MN17,Midtown-Midtown South,3807,1864600.41684,5693.03636727,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."


In [350]:
gdf["boro_ct2010"] = gdf["boro_ct2010"].astype(int)

In [351]:
#country_shapes = country_shapes.merge(country_names, on='iso_a3')
NYC_shape = gdf.merge(NYC_pop_data, on="boro_ct2010")

In [352]:
NYC_shape.shape

(132, 18)

In [353]:
NYC_shape.head()

,boro_code,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,the_geom,borough,census_tract,dcp_borough_code,fips_county_code,population,year
0,3,3000501,Brooklyn,I,000501,5.01,BK09,Brooklyn Heights-Cobble Hill,4004,901506.97904,4828.16991186,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,000501,3,47,4247,2010
1,3,3002000,Brooklyn,I,002000,20,BK32,Sunset Park West,4012,1620770.16274,5426.06727936,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,002000,3,47,1672,2010
2,3,3004100,Brooklyn,I,004100,41,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,1744094.3952,5283.85665969,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,004100,3,47,3598,2010
3,3,3005202,Brooklyn,I,005202,52.02,BK31,Bay Ridge,4013,1398134.42883,5190.71381772,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,005202,3,47,2481,2010
4,3,3005800,Brooklyn,E,005800,58,BK31,Bay Ridge,4013,2080303.3141,6098.16779894,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,005800,3,47,3152,2010


In [354]:
NYC_shape.population.dtype

dtype('int64')

In [355]:
NYC_shape.shape_area.dtype

dtype('O')

In [356]:
NYC_shape['shape_area'] = NYC_shape['shape_area'].convert_objects(convert_numeric=True)

/Applications/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [357]:
#NYC_shape.shape_area = NYC_shape.shape_area.astype(float)

In [358]:
NYC_shape.shape_area.dtype

dtype('float64')

In [359]:
NYC_shape.head()

,boro_code,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,the_geom,borough,census_tract,dcp_borough_code,fips_county_code,population,year
0,3,3000501,Brooklyn,I,000501,5.01,BK09,Brooklyn Heights-Cobble Hill,4004,9.015070e+05,4828.16991186,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,000501,3,47,4247,2010
1,3,3002000,Brooklyn,I,002000,20,BK32,Sunset Park West,4012,1.620770e+06,5426.06727936,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,002000,3,47,1672,2010
2,3,3004100,Brooklyn,I,004100,41,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,1.744094e+06,5283.85665969,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,004100,3,47,3598,2010
3,3,3005202,Brooklyn,I,005202,52.02,BK31,Bay Ridge,4013,1.398134e+06,5190.71381772,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,005202,3,47,2481,2010
4,3,3005800,Brooklyn,E,005800,58,BK31,Bay Ridge,4013,2.080303e+06,6098.16779894,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,005800,3,47,3152,2010


In [360]:
#from fiona.crs import from_epsg
#ct_shape.crs = from_epsg(4326) # epsg=4326: lat/on | 26918: NAD83/UTM zone 18N | epsg=2263 is US feet
#NYC_Area=ct_shape.to_crs(epsg=2263).geometry.area.sum()/(2.788*10**7) # from ft2 to mi2 
#print 'total NYC land area: %s (mi^2)'%str(NYC_Area)[:5] # from ft2 to mi2 #epsg=2263 is US feet

In [361]:
NYC_shape.crs = from_epsg(4326)

In [362]:
NYC_shape.crs

{'init': 'epsg:4326', 'no_defs': True}

In [363]:
# Population density by 1,000 sq feet
NYC_shape["pop_density"] = (NYC_shape.population/NYC_shape.shape_area)*1000

In [364]:
NYC_shape.head()

,boro_code,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,the_geom,borough,census_tract,dcp_borough_code,fips_county_code,population,year,pop_density
0,3,3000501,Brooklyn,I,000501,5.01,BK09,Brooklyn Heights-Cobble Hill,4004,9.015070e+05,4828.16991186,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,000501,3,47,4247,2010,4.711001
1,3,3002000,Brooklyn,I,002000,20,BK32,Sunset Park West,4012,1.620770e+06,5426.06727936,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,002000,3,47,1672,2010,1.031608
2,3,3004100,Brooklyn,I,004100,41,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,1.744094e+06,5283.85665969,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,004100,3,47,3598,2010,2.062962
3,3,3005202,Brooklyn,I,005202,52.02,BK31,Bay Ridge,4013,1.398134e+06,5190.71381772,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,005202,3,47,2481,2010,1.774507
4,3,3005800,Brooklyn,E,005800,58,BK31,Bay Ridge,4013,2.080303e+06,6098.16779894,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Brooklyn,005800,3,47,3152,2010,1.515164


In [365]:
#building_data = pd.read_json("https://data.cityofnewyork.us/resource/q3m4-ttp3.json", dtype={'census_tract': str}) 

In [366]:
path = "../Downloads/medianhouseholdincomecensustract.csv"
house_data = pd.read_csv(path, dtype={'TRACTCE10': str})
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,MSMOC_TOT_,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,22740,2264.889916,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,27804,2363.430717,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,28068,1260.718928,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,22620,1229.562591,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,20724,2209.874484,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234


In [367]:
house_data.columns

Index(['FID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'INTPTLAT10',
       'INTPTLON10', 'State', 'COUNTY', 'HH_COUNT', 'OHU_OWN', 'OHU_RENT',
       'PERC_OWN', 'PERC_RENT', 'MHI', 'MED_VAL', 'HOUSINGCOS', 'MCR_ANNUAL',
       'MSMOC_TOT_', 'TAX_RET', 'Avg_TransC', 'REtaxperow', 'DISP_INC',
       'energy_cos', 'REtax_ACS', 'AVG_TTL', 'LOCALNAME', 'Shape_Area'],
      dtype='object')

In [372]:
#house_data[house_data.TRACTCE10.str.contains("11")]

In [373]:
#house_data["boro_ct2010"] = house_data.TRACTCE10.str.ljust(5,'0')

In [374]:
house_data["boro_ct2010"] = house_data.TRACTCE10.str.zfill(6)

In [375]:
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_ct2010
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,2264.889916,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036,032400
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,2363.430717,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106,010100
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,1260.718928,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031,010200
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,1229.562591,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108,010300
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,2209.874484,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234,010400


Manhattan - New York County. 1
Bronx - Bronx County. 2
Brooklyn - Kings County. 3
Queens - Queens County. 4
Staten Island - Richmond County. 5

In [376]:
def boro(x):
    if x == "New York County":
        return 1
    if x == "Bronx County":
        return 2
    if x == "Kings County":
        return 3
    if x == "Queens County":
        return 4
    if x == "Richmond County":
        return 5

In [377]:
boro('New York County')

1

In [378]:
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_ct2010
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,2264.889916,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036,032400
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,2363.430717,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106,010100
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,1260.718928,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031,010200
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,1229.562591,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108,010300
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,2209.874484,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234,010400


In [379]:
house_data["boro_code"] = house_data.COUNTY.map({"New York County": "1", "Bronx County": "2", "Kings County": "3", \
                                           "Queens County": "4", "Richmond County": "5"})

In [380]:
house_data.boro_ct2010 = house_data.boro_code.astype(str) + house_data.boro_ct2010.astype(str)

In [382]:
#house_data[house_data.boro_code == "3"]

In [383]:
NYC_shape[NYC_shape.boro_ct2010 == 3074000]

,boro_code,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,the_geom,borough,census_tract,dcp_borough_code,fips_county_code,population,year,pop_density


In [384]:
#building_data["boro_ct2010"] = building_data.census_tract.str.ljust(5, '0')

In [385]:
#building_data["boro_ct2010"] = building_data.boro_ct2010.str.zfill(6)

In [386]:
#del building_data["great"]

In [387]:
#building_data.tail()

In [388]:
#building_data.boro_ct2010.dtype

In [389]:
#building_data["great"] = building_data.boro_ct2010.str.len() > 6

In [390]:
#building_data.shape

In [391]:
#building_data.borough.unique()

In [392]:
#building_data["boro_code"] = building_data.borough.map({"Manhattan": "1", "Bronx": "2", "Brooklyn": "3", \
                                          # "Queens": "4", "Staten Island": "5"})

In [393]:
#building_data["boro_ct2010"] = building_data["boro_code"].astype(str) + building_data["boro_ct2010"].astype(str)

In [394]:
#building_data.tail()

In [395]:
#NYC_shape merge with house data merge with building_data

In [396]:
NYC_shape.shape

(132, 19)

In [397]:
NYC_shape.boro_ct2010.dtype

dtype('int64')

In [398]:
NYC_shape.boro_ct2010 = NYC_shape.boro_ct2010.astype(str)

In [399]:
NYC_shape.boro_ct2010.dtype

dtype('O')

In [400]:
NYC_shape = NYC_shape.merge(house_data, on="boro_ct2010")

In [401]:
NYC_shape.shape

(132, 48)

In [402]:
NYC_shape.head()

,boro_code_x,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_code_y
0,3,3000501,Brooklyn,I,000501,5.01,BK09,Brooklyn Heights-Cobble Hill,4004,9.015070e+05,...,1737.071267,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,0.000009,3
1,3,3002000,Brooklyn,I,002000,20,BK32,Sunset Park West,4012,1.620770e+06,...,574.850160,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.000016,3
2,3,3004100,Brooklyn,I,004100,41,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,1.744094e+06,...,1374.638495,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.000017,3
3,3,3005202,Brooklyn,I,005202,52.02,BK31,Bay Ridge,4013,1.398134e+06,...,1582.632893,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,0.000015,3
4,3,3005800,Brooklyn,E,005800,58,BK31,Bay Ridge,4013,2.080303e+06,...,1713.269708,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.000021,3


In [403]:
house_data.boro_ct2010 = house_data.boro_ct2010.astype(str)

In [404]:
house_data.boro_ct2010.dtype

dtype('O')

In [405]:
house_data.head()

,FID,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,INTPTLAT10,INTPTLON10,State,COUNTY,HH_COUNT,...,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_ct2010,boro_code
0,0,34,17,32400,34017032400,40.792844,-74.013482,New Jersey,Hudson County,2187,...,4145.566539,4966.370331,12232.00950,1773.280152,7735,5007.444405,West New York Town,0.000036,nan032400,NaN
1,1,34,17,10100,34017010100,40.691559,-74.110913,New Jersey,Hudson County,2255,...,5969.286255,4490.984109,22077.84024,1876.844806,7917,6146.888195,Bayonne City,0.000106,nan010100,NaN
2,2,34,17,10200,34017010200,40.682103,-74.104573,New Jersey,Hudson County,1218,...,6014.489564,3339.224580,20488.60531,2167.175106,7692,6193.436421,Bayonne City,0.000031,nan010200,NaN
3,3,34,17,10300,34017010300,40.672439,-74.081016,New Jersey,Hudson County,1093,...,4798.787758,3793.386763,15514.61337,2163.964082,7831,4941.564294,Bayonne City,0.000108,nan010300,NaN
4,4,34,17,10400,34017010400,40.670599,-74.089940,New Jersey,Hudson County,1571,...,7611.970058,4478.110461,35131.00534,2114.480847,7918,7838.446154,Bayonne City,0.000234,nan010400,NaN


In [406]:
NYC_shape.head()

,boro_code_x,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_code_y
0,3,3000501,Brooklyn,I,000501,5.01,BK09,Brooklyn Heights-Cobble Hill,4004,9.015070e+05,...,1737.071267,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,0.000009,3
1,3,3002000,Brooklyn,I,002000,20,BK32,Sunset Park West,4012,1.620770e+06,...,574.850160,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.000016,3
2,3,3004100,Brooklyn,I,004100,41,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,1.744094e+06,...,1374.638495,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.000017,3
3,3,3005202,Brooklyn,I,005202,52.02,BK31,Bay Ridge,4013,1.398134e+06,...,1582.632893,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,0.000015,3
4,3,3005800,Brooklyn,E,005800,58,BK31,Bay Ridge,4013,2.080303e+06,...,1713.269708,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.000021,3


In [407]:
#building_data.boro_ct2010.dtype

In [408]:
#NYC_shape = NYC_shape.merge(building_data, on="boro_ct2010", how="outer")

In [409]:
NYC_shape.shape

(132, 48)

In [410]:
house_data.columns

Index(['FID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'INTPTLAT10',
       'INTPTLON10', 'State', 'COUNTY', 'HH_COUNT', 'OHU_OWN', 'OHU_RENT',
       'PERC_OWN', 'PERC_RENT', 'MHI', 'MED_VAL', 'HOUSINGCOS', 'MCR_ANNUAL',
       'MSMOC_TOT_', 'TAX_RET', 'Avg_TransC', 'REtaxperow', 'DISP_INC',
       'energy_cos', 'REtax_ACS', 'AVG_TTL', 'LOCALNAME', 'Shape_Area',
       'boro_ct2010', 'boro_code'],
      dtype='object')

In [411]:
#building_data.columns

In [412]:
# household_density per 1000 sq feet by census tract
NYC_shape["household_density"] = (NYC_shape.HH_COUNT/NYC_shape.shape_area) *1000

In [413]:
NYC_shape.head()

,boro_code_x,boro_ct2010,boro_name,cdeligibil,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,...,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,Shape_Area,boro_code_y,household_density
0,3,3000501,Brooklyn,I,000501,5.01,BK09,Brooklyn Heights-Cobble Hill,4004,9.015070e+05,...,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,0.000009,3,2.084288
1,3,3002000,Brooklyn,I,002000,20,BK32,Sunset Park West,4012,1.620770e+06,...,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.000016,3,0.269625
2,3,3004100,Brooklyn,I,004100,41,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,1.744094e+06,...,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.000017,3,0.876099
3,3,3005202,Brooklyn,I,005202,52.02,BK31,Bay Ridge,4013,1.398134e+06,...,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,0.000015,3,1.009202
4,3,3005800,Brooklyn,E,005800,58,BK31,Bay Ridge,4013,2.080303e+06,...,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.000021,3,0.755659


In [414]:
NYC_shape["boro_code"] = NYC_shape.boro_code_x

In [415]:
del NYC_shape["cdeligibil"]
del NYC_shape["borough"]
del NYC_shape["census_tract"]
del NYC_shape["dcp_borough_code"]
del NYC_shape["FID"]
del NYC_shape["STATEFP10"]
del NYC_shape["COUNTYFP10"]
del NYC_shape["TRACTCE10"]
del NYC_shape["GEOID10"]
del NYC_shape["Shape_Area"]
del NYC_shape["boro_code_y"]
del NYC_shape["boro_code_x"]

In [416]:
NYC_shape.head()

,boro_ct2010,boro_name,ct2010,ctlabel,ntacode,ntaname,puma,shape_area,shape_leng,the_geom,...,TAX_RET,Avg_TransC,REtaxperow,DISP_INC,energy_cos,REtax_ACS,AVG_TTL,LOCALNAME,household_density,boro_code
0,3000501,Brooklyn,000501,5.01,BK09,Brooklyn Heights-Cobble Hill,4004,9.015070e+05,4828.16991186,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",...,1737.071267,7205.977121,2600.965286,36501.21653,1412.983117,4711,17116.407730,Brooklyn Heights,2.084288,3
1,3002000,Brooklyn,002000,20,BK32,Sunset Park West,4012,1.620770e+06,5426.06727936,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",...,574.850160,3064.116727,580.101530,13647.01968,2061.361323,1700,2968.716870,Bush Terminal,0.269625,3
2,3004100,Brooklyn,004100,41,BK38,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,4004,1.744094e+06,5283.85665969,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",...,1374.638495,8024.382252,2001.736607,39450.88208,1690.291069,4224,15948.483090,Boerum Hill,0.876099,3
3,3005202,Brooklyn,005202,52.02,BK31,Bay Ridge,4013,1.398134e+06,5190.71381772,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",...,1582.632893,5761.039594,2039.785814,33727.02430,1585.245533,2937,9382.433070,Bay Ridge,1.009202,3
4,3005800,Brooklyn,005800,58,BK31,Bay Ridge,4013,2.080303e+06,6098.16779894,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",...,1713.269708,4568.334204,3609.383943,24360.97565,1599.484797,4026,7439.992247,Bay Ridge,0.755659,3


In [417]:
writer = pd.ExcelWriter('MEGA_DATASET.xlsx')
NYC_shape.to_excel(writer)
writer.save()

In [418]:
# geenrate points for wifi data and sjoin with shape maga dataset
# generate regression model